# Global Imports

In [ ]:
import os
import pprint
import sys
from pathlib import Path as Pt
from typing import Dict, List, Optional, Set, Tuple, Union

import numpy as np

from utils_002 import Template, gen_xyz, load_cfg, prep_cuboid

# MK Steps Limit Recalculation

In [ ]:
def process_directory(dir_path: Pt) -> Tuple[Dict, Optional[np.ndarray]]:
    try:
        path_cfg = dir_path / "InitSettings.ini"
        path_history = dir_path / "sim_history.txt"

        if not (path_cfg.exists() and path_history.exists()):
            raise FileNotFoundError(f"Required files missing in {dir_path}")

        data_ini = load_cfg(path_to_config=path_cfg)
        run_id = dir_path.name.split("_")[0]
        data_ini["id"] = run_id

        history_txt = path_history.read_text(encoding="utf-8")
        history_data = np.array(
            [
                *map(
                    lambda line: [*map(float, line.split(":"))],
                    history_txt.splitlines(),
                )
            ]
        )
        # print(history_data)
        (
            n_gas_history,
            n_crystal_history,
            concentration_history,
            delta_gibbs_history,
            energy_change_history,
            crystal_sx_history,
            crystal_sy_history,
            crystal_sz_history,
            mk_step_history,
        ) = history_data

        # print(f"n_gas_history: {n_gas_history}")
        # print(f"n_crystal_history: {n_crystal_history}")
        # print(f"concentration_history: {concentration_history}")
        # print(f"delta_gibbs_history: {delta_gibbs_history}")
        # print(f"energy_change_history: {energy_change_history}")
        # print(f"crystal_sx_history: {crystal_sx_history}")
        # print(f"crystal_sy_history: {crystal_sy_history}")
        # print(f"crystal_sz_history: {crystal_sz_history}")
        # print(f"mk_step_history: {mk_step_history}")

        return delta_gibbs_history[0], concentration_history[0], mk_step_history[-1]



    except Exception as e:
        print(f"Error processing directory {dir_path}: {str(e)}")
        raise


base_folder = Pt(
    r"20250722T204157_6998"
)

res = []

try:
    for root, _, files in os.walk(base_folder):
        if {"InitSettings.ini", "sim_history.txt"}.issubset(files):
            print(f"Processing directory: {root}")
            dg, c0, mk = process_directory(Pt(root))
            res.append([dg, c0, mk])
except Exception as e:
    print(f"Error in main loop: {str(e)}")
    raise

res = np.array(res)
dgs, c0s, mks = res.T

In [ ]:
N = 10

recalculated_write_i = (mks / (N + 1)).astype(int) + 1
recalculated_write_i

In [ ]:
# mks[15], mks[15] / (N + 1), recalculated_write_i[15], recalculated_write_i[15] * (N + 1)

In [ ]:
data = np.vstack([dgs, c0s, recalculated_write_i]).T
data[:5]

# Preparation of the Combinations file

In [ ]:
cfg = {
    "DirPrefix": {"v": "Base", "f": "{}"},
    "Seed": {"v": 1012, "f": "{}"},
    "Sx": {"v": 51, "f": "{}"},
    "Sy": {"v": 51, "f": "{}"},
    "Sz": {"v": 51, "f": "{}"},
    "Px": {"v": False, "f": "{}"},
    "Py": {"v": False, "f": "{}"},
    "Pz": {"v": False, "f": "{}"},
    "T": {"v": 300.0, "f": "{:.5f}"},
    "Ax": {"v": 5.85e-10, "f": "{:.5e}"},
    "Ay": {"v": 1.78e-10, "f": "{:.5e}"},
    "Az": {"v": 4.41e-10, "f": "{:.5e}"},
    "g100": {"v": 0.41, "f": "{:.5f}"},
    "g010": {"v": 0.54, "f": "{:.5f}"},
    "g001": {"v": 0.22, "f": "{:.5f}"},
    "mode": {"v": 1, "f": "{}"},
    "dg": {"v": 0.0, "f": "{:.5e}"},
    "C_eq": {"v": 9.58767e-08, "f": "{:.5e}"},
    "C0": {"v": 9.58767e-08, "f": "{:.5e}"},
    "N_tot": {"v": 5e12, "f": "{:.5e}"},
    "N0_cr": {"v": -1.0, "f": "{:.5e}"},
    "p_b": {"v": -1.0, "f": "{:.5f}"},
    "p_pow": {"v": 1.0, "f": "{:.5f}"},
    "AddI": {"v": 1, "f": "{}"},
    "AddFrom": {"v": 1, "f": "{}"},
    "RemI": {"v": 1, "f": "{}"},
    "RemFrom": {"v": 1, "f": "{}"},
    "LoadPrev": {"v": 0, "f": "{}"},
    "StepLim": {"v": 1_000_000, "f": "{}"},
    "PrintI": {"v": 100_000, "f": "{}"},
    "WriteI": {"v": 10_000, "f": "{}"},
    "Version": {"v": "6.0.0", "f": "{}"},
    "ExeFileName": {"v": "M1_v6.0.0.exe", "f": "{}"},
    "comments_adds": {"v": "", "f": "{}"},
}

tpl = Template(cfg, template_path="WorkTemplate.txt")
print(f"\nData (Compact):\n{tpl.template_compact()}")

In [ ]:
Ax = 5.85E-10
Ay = 1.78E-10
Az = 4.41E-10

# A1x = Ax / (2 ** (1 / 3))
# A1y = Ay / (2 ** (1 / 3))
# A1z = Az / (2 ** (1 / 3))

# A1x = Ax / (7 ** (1 / 3))
# A1y = Ay / (7 ** (1 / 3))
# A1z = Az / (7 ** (1 / 3))

A1x = Ax / (14 ** (1 / 3))
A1y = Ay / (14 ** (1 / 3))
A1z = Az / (14 ** (1 / 3))

g100 = 0.41
g010 = 0.54
g001 = 0.22

Ω = A1x * A1y * A1z

EX = 2 * g100 * A1y * A1z
EY = 2 * g010 * A1z * A1x
EZ = 2 * g001 * A1x * A1y

Eisol = EX + EY + EZ
E3 = EY + EX
E2 = EZ + EY
E1 = EZ + EX

kB = 1.380649e-23
T = 300.0
kT = kB * T
Ceq = np.exp(- Eisol / kT)

print(
    """
Sizes:
    Ax: {:.5e},    Ay: {:.5e},    Az: {:.5e}
   A1x: {:.5e},   A1y: {:.5e},   A1z: {:.5e}

Volume:
   Ω = A1x * A1y * A1z = {:.5e} * {:.5e} * {:.5e} = {:.5e}

Surface tensions:
   g100: {:.5e}, g010: {:.5e}, g001: {:.5e}

Energys:
1)
   EX = 2.0 * g100 * A1y * A1z = 2.0 * {:.5e} * {:.5e} * {:.5e} = {:.5e}
   EY = 2.0 * g010 * A1z * A1x = 2.0 * {:.5e} * {:.5e} * {:.5e} = {:.5e}
   EZ = 2.0 * g001 * A1x * A1y = 2.0 * {:.5e} * {:.5e} * {:.5e} = {:.5e}

2)
   Eisol = EX + EY + EZ = {:.5e} + {:.5e} + {:.5e} = {:.5e}

3)
   E3 = EY + EX = {:.5e} + {:.5e} = {:.5e}
   E2 = EZ + EY = {:.5e} + {:.5e} = {:.5e}
   E1 = EZ + EX = {:.5e} + {:.5e} = {:.5e}

kB = {:.5e}; T = {:.5f};
kT = kB * T = {:.5e} * {:.5f} = {:.5e};
Ceq = exp(- Eisol / kT) = exp(- {:.5e} / {:.5e} ) = {:.5e};
""".format(
        Ax, Ay, Az,
        A1x, A1y, A1z,
        A1x, A1y, A1z,
        Ω,
        g100, g010, g001,
        g100, A1y, A1z, EX,
        g010, A1z, A1x, EY,
        g001, A1x, A1y, EZ,
        EX, EY, EZ, Eisol,
        EY, EX, E3,
        EZ, EY, E2,
        EZ, EX, E1,
        kB, T,
        kB, T, kT,
        Eisol, kT, Ceq,
    ).strip()
)

In [ ]:
# range_01_0_to_E1 = np.linspace(0.0, E1, 11)
# range_01_E1_to_E2 = np.linspace(E1, E2, 11)
# range_01_E2_to_E3 = np.linspace(E2, E3, 11)
# range_01_E3_to_Eisol = np.linspace(E3, Eisol, 11)

# dg_s_positive = np.hstack(
#     [
#         # range_01_0_to_E1,
#         # range_01_E1_to_E2,
#         # range_01_E2_to_E3,
#         # range_01_E3_to_Eisol,
#         # ======================
#         range_01_0_to_E1,
#         range_01_E1_to_E2[1:],
#         range_01_E2_to_E3[1:],
#         range_01_E3_to_Eisol[1:],
#     ]
# )

# dg_s_negative = (-1.0 * dg_s_positive)[::-1]
# dg_s_negative[np.abs(dg_s_negative) <= 1e-32] = 0.0

# dg_s_full = np.hstack(
#     [
#         dg_s_negative,
#         dg_s_positive[1:],
#     ]
# )

# # dg_s = np.arange(0.0, Eisol + 1e-21, 1e-21)

# dg_s = dg_s_positive
# # dg_s = dg_s_negative
# # dg_s = dg_s_full

# dg_s = np.around(dg_s, 32)
# dg_s

In [ ]:
tpl.set_item_v("Version", "6.0.0")
tpl.set_item_v("ExeFileName", "M1_v6.0.0.exe")

tpl.set_item_v("DirPrefix", "RN_EX_E1")

tpl.set_item_v("Sx", 50)
tpl.set_item_v("Sy", 50)
tpl.set_item_v("Sz", 50)
tpl.set_item_v("Px", False)
tpl.set_item_v("Py", False)
tpl.set_item_v("Pz", False)

tpl.set_item_v("Ax", A1x)
tpl.set_item_v("Ay", A1y)
tpl.set_item_v("Az", A1z)

tpl.set_item_v("LoadPrev", -1)

tpl.set_item_v("StepLim", 100_000_000)
tpl.set_item_v("PrintI", 1_000_000)
tpl.set_item_v("WriteI", 100_000)

comments_adds_t = """
Ax = 5.85E-10 / (14 ^ (1 / 3))
Ay = 1.78E-10 / (14 ^ (1 / 3))
Az = 4.41E-10 / (14 ^ (1 / 3))
""".strip()

prepared_combs_lines = []
for (dg, WriteI) in data:
    tpl.set_item_v("dg", dg)
    tpl.set_item_v("WriteI", int(WriteI))
    tpl.set_item_v("comments_adds", comments_adds_t.strip().replace("\n", "\\n\\"))

    comb_str = tpl.template_compact(splitter=" +| ")

    prepared_combs_lines.append(comb_str)

Pt("WorkCombs.txt").write_text(
    "\n".join(prepared_combs_lines).strip(), encoding="utf-8"
)

In [ ]:
tpl.set_item_v("Version", "6.0.0")
tpl.set_item_v("ExeFileName", "M1_v6.0.0.exe")

tpl.set_item_v("DirPrefix", "Mode2")

tpl.set_item_v("Sx", 50)
tpl.set_item_v("Sy", 50)
tpl.set_item_v("Sz", 50)
tpl.set_item_v("Px", False)
tpl.set_item_v("Py", False)
tpl.set_item_v("Pz", False)

tpl.set_item_v("Ax", A1x)
tpl.set_item_v("Ay", A1y)
tpl.set_item_v("Az", A1z)

tpl.set_item_v("mode", 2)
tpl.set_item_v("dg", 0.0)
tpl.set_item_v("C_eq", 9.58767e-8)
# tpl.set_item_v("N_tot", 1e22)
tpl.set_item_v("N0_cr", -1.0)
tpl.set_item_v("p_b", -1.0)

tpl.set_item_v("LoadPrev", -1)

tpl.set_item_v("StepLim", 100_000_000)
tpl.set_item_v("PrintI", 1_000_000)
tpl.set_item_v("WriteI", 100_000)

comments_adds_t = """
Ax = 5.85E-10 / (14 ^ (1 / 3))
Ay = 1.78E-10 / (14 ^ (1 / 3))
Az = 4.41E-10 / (14 ^ (1 / 3))
""".strip()

prepared_combs_lines = []
for (dg, c0, WriteI) in data:
    tpl.set_item_v("dg", dg)
    tpl.set_item_v("N_tot", 1e14)
    tpl.set_item_v("C0", c0)
    tpl.set_item_v("WriteI", int(WriteI))
    tpl.set_item_v("comments_adds", comments_adds_t.strip().replace("\n", "\\n\\"))

    comb_str = tpl.template_compact(splitter=" +| ")

    prepared_combs_lines.append(comb_str)

Pt("WorkCombs.txt").write_text(
    "\n".join(prepared_combs_lines).strip(), encoding="utf-8"
)